# Churn case Part 1

In [1]:
import pandas as pd

## Reading all CSV files with Pandas

In [2]:
customer = pd.read_csv("customer.csv")
customer.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents
0,3668-QPYBK,Male,No,No,No
1,9237-HQITU,Female,No,No,Yes
2,9305-CDSKC,Female,No,No,Yes
3,7892-POOKP,Female,No,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes


In [3]:
cust_loc = pd.read_csv("cust_loc.csv")
cust_loc.head()

,Cust_ID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,California,34.039224,-118.266293,90015


In [4]:
cust_services = pd.read_csv("cust_services.csv")
cust_services.head()

,Cust_ID,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes


In [5]:
cust_account = pd.read_csv("cust_account.csv")
cust_account.head()

,Account_id,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges
0,3668-QPYBK,2,Month-to-month,Mailed check,Yes,53.85,108.15
1,9237-HQITU,2,Month-to-month,Electronic check,Yes,70.70,151.65
2,9305-CDSKC,8,Month-to-month,Electronic check,Yes,99.65,820.5
3,7892-POOKP,28,Month-to-month,Electronic check,Yes,104.80,3046.05
4,0280-XJGEX,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.3


In [6]:
cust_churn = pd.read_csv("cust_churn.csv")
cust_churn.head()

,Id,Churn
0,3668-QPYBK,Yes
1,9237-HQITU,Yes
2,9305-CDSKC,Yes
3,7892-POOKP,Yes
4,0280-XJGEX,Yes


## Create database and tables

In [7]:
import sqlite3

In [8]:
conn = sqlite3.connect("churn.db")
cur= conn.cursor()

In [12]:
# Create customer table
customer.to_sql("customer",conn,if_exists= "replace",index=False)

7043

In [17]:
# Create customer location table
cust_loc.to_sql("cust_loc",conn,if_exists= "replace",index=False)

7043

In [62]:
# Create customer services table
cust_services.to_sql("cust_services",conn,if_exists= "replace",index=False)

7043

In [19]:
# Create customer account table
cust_account.to_sql("cust_account",conn,if_exists= "replace",index=False)

7043

In [65]:
# Create customer churn table
cust_churn.to_sql("cust_churn",conn,if_exists= "replace",index=False)

7043

# Information about the tables

In [11]:
def table_info(conn,cursor):
    """
    prints out all of the columns of every table in DB
    
    conn: database connection object
    cursor: cursor object
    """
    tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    for table_name in tables:
        table_name= table_name[0]
        table= pd.read_sql_query("SELECT * FROM {} LIMIT 0".format(table_name),conn)
        print(table_name)
        for col in table.columns:
            print("\t"+ col)
            print()

In [12]:
table_info(conn,cur)

customer
	CustomerID

	Gender

	SeniorCitizen

	Partner

	Dependents

cust_loc
	CustomerId

	State

	Latitude

	Longitude

	ZipCode

cust_services
	Cust_ID

	PhoneService

	MultipleLines

	InternetService

	OnlineSecurity

	OnlineBackup

	DeviceProtection

	TechSupport

	StreamingTV

	StreamingMovies

cust_account
	Account_id

	Tenure

	Contract

	PaymentMethod

	PaperlessBilling

	MonthlyCharges

	TotalCharges

cust_churn
	Id

	Churn



**To create a ERD Diagram for Database**

**Use https://dbdiagram.io/d**

## Join all tables

In [9]:
cur.execute("""
ALTER TABLE cust_loc
RENAME COLUMN Cust_ID TO CustomerId
""")

In [18]:
cur.execute("""
SELECT * FROM customer
inner join cust_loc
using (CustomerID)
""")
cur.fetchone()

('3668-QPYBK',
 'Male',
 'No',
 'No',
 'No',
 'California',
 33.964131,
 -118.272783,
 90003)

In [20]:
cust_loc = cust_loc.rename(columns ={"Cust_ID":"CustomerID"})

In [21]:
cust_loc.head()

,CustomerID,State,Latitude,Longitude,ZipCode
0,3668-QPYBK,California,33.964131,-118.272783,90003
1,9237-HQITU,California,34.059281,-118.307420,90005
2,9305-CDSKC,California,34.048013,-118.293953,90006
3,7892-POOKP,California,34.062125,-118.315709,90010
4,0280-XJGEX,California,34.039224,-118.266293,90015


In [30]:
# Rename remaining columns from tables

In [31]:
cust_services = cust_services.rename(columns ={"Cust_ID":"CustomerID"})
cust_account = cust_account.rename(columns ={"Account_id":"CustomerID"})
cust_churn = cust_churn.rename(columns ={"Id":"CustomerID"})

In [32]:
dfs_to_join = [customer, cust_loc, cust_services, cust_account, cust_churn]

In [34]:
churn_all= pd.concat(dfs_to_join, join="inner", axis=1)

In [35]:
churn_all.columns.duplicated()

array([False, False, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
        True, False])

In [38]:
churn_all= churn_all.loc[:, ~churn_all.columns.duplicated()]

In [39]:
churn_all.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic check,Yes,99.65,820.5,Yes
3,7892-POOKP,Female,No,Yes,Yes,California,34.062125,-118.315709,90010,Yes,...,Yes,Yes,Yes,28,Month-to-month,Electronic check,Yes,104.80,3046.05,Yes
4,0280-XJGEX,Male,No,No,Yes,California,34.039224,-118.266293,90015,Yes,...,No,Yes,Yes,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.3,Yes


In [40]:
churn_all.to_csv('churn_all.csv')

In [42]:
churn_all.to_sql('churn_all',conn, if_exists= 'replace', index=False)

7043